In [29]:
#|export
"""
Run simply with
$ pytest
"""

import os
import subprocess
import numpy as np
import torch
import struct

from export import export_model, export_modelq8
from train import load

#this is predicted wrong on fastai resnet18
file_path = "test/data/imagenette2/val_transformed/0/113"

In [30]:
#|export
# calculate reference values using Python model
model = load("resnet18")
with open(file_path, "rb") as f:
    sizeof_float, nch, h, w = 4, 3, 224, 224
    image = torch.tensor(struct.unpack("f"*(nch*h*w), f.read(sizeof_float*nch*h*w))).view(1, nch,h,w)
    ref = model(image).detach()
    ref = torch.nn.functional.softmax(ref, dim=1).view(-1).numpy() # python model output

In [31]:
#|export
def execute(command):
    d = "test_outputs"
    os.makedirs(d, exist_ok=True)
    err = os.path.join(d, "err.txt")
    out = os.path.join(d, "stdout.txt")
    with open(err, mode="w") as fe:
        with open(out, mode="w") as fo:
            proc = subprocess.Popen(command, stdout=fo, stderr=fe)
            proc.wait()
    res = np.loadtxt(out)

    return res

def test_runfiles(quantized=True, file_path=file_path):
    """ test run.c and runq.c works with an acceptable tolerance """
    # run vanilla model in test mode
    export_model("resnet18")
    command = ["./run", "model.bin", file_path]
    res = execute(command)
    assert np.allclose(res, ref, atol=1e-5, rtol=0), "run.c: Probabilities are not close."

    if quantized:
        # run quantized model test with group size = 1 and 2 in test mode
        export_modelq8(file_path="modelq8_1.bin", gs=1)
        resq1 = execute(["./runq", "modelq8_1.bin", file_path])
        assert np.allclose(resq1, ref, atol=1e-5, rtol=0), "runq.c (group size = 1): Probabilities are not close."

        export_modelq8(file_path="modelq8_2.bin", gs=2)
        resq2 = execute(["./runq", "modelq8_2.bin", file_path])
        assert np.allclose(resq2, ref, atol=1e-2, rtol=0), "runq.c (group size = 2):Probabilities are not close."

    print("Done")

In [32]:
test_runfiles(False)

wrote model.bin
Done


In [26]:
def hook_fn(module, input, output):
    print(output)
    print(output.shape)
    return output


model = load("resnet18")
last_resnet_layer = model[0][7]
pool_layer = model[1][0]
bn_1_layer = model[1][2]
ln1_layer = model[1][4]
bn2_layer = model[1][6]
ln2_layer = model[1][8]

file_path = "data/imagenette2/val_transformed/0/2"
with open(file_path, "rb") as f:
    sizeof_float, nch, h, w = 4, 3, 224, 224
    image = torch.tensor(struct.unpack("f"*(nch*h*w), f.read(sizeof_float*nch*h*w))).view(1, nch,h,w)
    hook = ln2_layer.register_forward_hook(hook_fn)
    out = model(image).detach()
    output = torch.nn.functional.softmax(out, dim=1).view(-1).numpy()
    print(np.argmax(output))
    hook.remove()

tensor([[ 8.0158,  1.4867, -2.1652, -2.6870, -0.2114, -2.4135, -2.1969, -1.0151,
          3.1303,  7.8046]], grad_fn=<MmBackward0>)
torch.Size([1, 10])
0
